In [ ]:
#pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch==0.4.1.post2 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==0.4.1.post2


In [1]:
import pdb
import argparse
import numpy as np
import numpy.random as npr
import time
import os
import sys
import pickle
import ssl 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR

from torchvision.utils import make_grid
from torchvision import datasets, transforms

from Cutout.util.misc import CSVLogger
from Cutout.util.cutout import Cutout

from Cutout.model.resnet import ResNet18
from Cutout.model.wide_resnet import WideResNet

In [17]:
# Format time for printing purposes
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

##### Noisy

In [18]:
# Introduce Gaussian noise to noise_percentage of image pixels
def noisy(image, noise_percentage, noise_std):
    row, col, ch = image.shape
    num_corrupt = int(np.floor(noise_percentage * row * col / 100))

    # Randomly choose pixels to add noise to
    xy_coords = np.random.choice(row * col, num_corrupt, replace=False)
    chan_coords = np.random.choice(ch, num_corrupt, replace=True)
    xy_coords = np.unravel_index(xy_coords, (row, col))

    out = np.copy(image)

    mean = 120

    # Add randomly generated Gaussian noise to pixels
    for coord in range(num_corrupt):
        noise = np.random.normal(mean, noise_std, 1)
        out[xy_coords[0][coord], xy_coords[1][coord],
            chan_coords[coord]] += noise

    return out

### Arguments

In [33]:
#define args 
args = {
    'dataset': 'cifar10', 
    'model': 'resnet18', 
    'batch_size': 128, 
    'epochs': 10, 
    'learning_rate': 0.1, 
    'data_augmentation': True, 
    'cutout': False, 
    'cuda':True,
    'n_holes': 1, 
    'length': 16, 
    'no_cuda': False, 
    'seed': 1, 
    'sorting_file': 'none', 
    'remove_n': 0, 
    'keep_lowest_n': 0, 
    'remove_subsample': 0, 
    'noise_percent_labels': 0, 
    'noise_percent_pixels': 0, 
    'noise_std_pixels': 0, 
    'optimizer': 'sgd', 
    'input_dir': 'cifar10_results/', 
    'output_dir': 'cifar10_results',
    'data_augmentation': False,
 }


save_fname = '__'

### Train

In [19]:
# Train model for one epoch
#
# example_stats: dictionary containing statistics accumulated over every presentation of example
#
def train(args, model, device, trainset, model_optimizer, epoch,
          example_stats):
    train_loss = 0.
    correct = 0.
    total = 0.

    model.train()

    # Get permutation to shuffle trainset
    trainset_permutation_inds = npr.permutation(
        np.arange(len(trainset.targets)))

    print('\n=> Training Epoch #%d' % (epoch))

    batch_size = args['batch_size']
    for batch_idx, batch_start_ind in enumerate(
            range(0, len(trainset.targets), batch_size)):

        # Get trainset indices for batch
        batch_inds = trainset_permutation_inds[batch_start_ind:
                                               batch_start_ind + batch_size]

        # Get batch inputs and targets, transform them appropriately
        transformed_trainset = []
        for ind in batch_inds:
            transformed_trainset.append(trainset.__getitem__(ind)[0])
        inputs = torch.stack(transformed_trainset)
        targets = torch.LongTensor(
            np.array(trainset.targets)[batch_inds].tolist())

        # Map to available device
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward propagation, compute loss, get predictions
        model_optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        _, predicted = torch.max(outputs.data, 1)

        # Update statistics and loss
        acc = predicted == targets
        for j, index in enumerate(batch_inds):

            # Get index in original dataset (not sorted by forgetting)
            index_in_original_dataset = train_indx[index]

            # Compute missclassification margin
            output_correct_class = outputs.data[j, targets[j].item()]
            sorted_output, _ = torch.sort(outputs.data[j, :])
            if acc[j]:
                # Example classified correctly, highest incorrect class is 2nd largest output
                output_highest_incorrect_class = sorted_output[-2]
            else:
                # Example misclassified, highest incorrect class is max output
                output_highest_incorrect_class = sorted_output[-1]
            margin = output_correct_class.item(
            ) - output_highest_incorrect_class.item()

            # Add the statistics of the current training example to dictionary
            index_stats = example_stats.get(index_in_original_dataset,
                                            [[], [], []])
            index_stats[0].append(loss[j].item())
            index_stats[1].append(acc[j].sum().item())
            index_stats[2].append(margin)
            example_stats[index_in_original_dataset] = index_stats

        # Update loss, backward propagate, update optimizer
        loss = loss.mean()
        train_loss += loss.item()
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        loss.backward()
        model_optimizer.step()

        sys.stdout.write('\r')
        sys.stdout.write(
            '| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%' %
            (epoch, args['epochs'], batch_idx + 1,
             (len(trainset) // batch_size) + 1, loss.item(),
             100. * correct.item() / total))
        sys.stdout.flush()

        # Add training accuracy to dict
        index_stats = example_stats.get('train', [[], []])
        index_stats[1].append(100. * correct.item() / float(total))
        example_stats['train'] = index_stats

### Test

In [34]:
# Evaluate model predictions on heldout test data
#
# example_stats: dictionary containing statistics accumulated over every presentation of example
#
def test(epoch, model, device, example_stats):
    global best_acc
    test_loss = 0.
    correct = 0.
    total = 0.
    test_batch_size = 32

    model.eval()

    for batch_idx, batch_start_ind in enumerate(
            range(0, len(test_dataset.targets), test_batch_size)):

        # Get batch inputs and targets
        transformed_testset = []
        for ind in range(
                batch_start_ind,
                min(
                    len(test_dataset.targets),
                    batch_start_ind + test_batch_size)):
            transformed_testset.append(test_dataset.__getitem__(ind)[0])
        inputs = torch.stack(transformed_testset)
        targets = torch.LongTensor(
            np.array(
                test_dataset.targets)[batch_start_ind:batch_start_ind +
                                          test_batch_size].tolist())

        # Map to available device
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward propagation, compute loss, get predictions
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss = loss.mean()
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Add test accuracy to dict
    acc = 100. * correct.item() / total
    index_stats = example_stats.get('test', [[], []])
    index_stats[1].append(100. * correct.item() / float(total))
    example_stats['test'] = index_stats
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %
          (epoch, loss.item(), acc))

    # Save checkpoint when best model
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' % (acc))
        state = {
            'acc': acc,
            'epoch': epoch,
        }
        save_point = os.path.join(args['output_dir'], 'checkpoint', args['dataset'])
        os.makedirs(save_point, exist_ok=True)
        torch.save(state, os.path.join(save_point, save_fname + '.t7'))
        best_acc = acc

### Dataset Processing

In [5]:
#prevents ssl certificate errors 
ssl._create_default_https_context = ssl._create_unverified_context

# Image Preprocessing
normalize = transforms.Normalize(
    mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
    std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# define train transform 
train_transform = transforms.Compose([])
if args['data_augmentation']:
    train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
    train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
if args['cutout']:
    train_transform.transforms.append(
        Cutout(n_holes=args.n_holes, length=args.length))

test_transform = transforms.Compose([transforms.ToTensor(), normalize])

# Check for output dir 
os.makedirs(args['output_dir'], exist_ok=True)

# Load the appropriate train and test datasets
if args['dataset'] == 'cifar10':
    num_classes = 10
    train_dataset = datasets.CIFAR10(
        root='data',
        train=True,
        transform=train_transform,
        download=True)

    test_dataset = datasets.CIFAR10(
        root='data',
        train=False,
        transform=test_transform,
        download=True)
elif args['dataset'] == 'cifar100':
    num_classes = 100
    train_dataset = datasets.CIFAR100(
        root='data',
        train=True,
        transform=train_transform,
        download=True)

    test_dataset = datasets.CIFAR100(
        root='data',
        train=False,
        transform=test_transform,
        download=True)

train_indx = npr.permutation(np.arange(len(
        train_dataset.targets)))[:len(train_dataset.targets) -
                                      0]

train_dataset.data = train_dataset.data[train_indx, :, :, :]
train_dataset.targets = np.array(
    train_dataset.targets)[train_indx].tolist()

print('Trainset of ' + str(len(train_dataset.targets)) + ' examples')


100%|██████████| 170M/170M [00:34<00:00, 4.98MB/s] 


Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified
Trainset of 50000 examples


### Model Setup

In [35]:
#device settings
args['cuda'] = args['cuda'] and torch.cuda.is_available()
use_cuda = args['cuda']
device = torch.device("cuda" if use_cuda else "cpu")
cudnn.benchmark = True  # Should make training go faster for large models

# Set random seed for initialization
torch.manual_seed(args['seed'])
if args['cuda']:
    torch.cuda.manual_seed(args['seed'])
npr.seed(args['seed'])

# Image Preprocessing
normalize = transforms.Normalize(
    mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
    std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# Setup model
if args['model'] == 'resnet18':
    model = ResNet18(num_classes=num_classes)
elif args['model'] == 'wideresnet':
        model = WideResNet(
            depth=28, num_classes=num_classes, widen_factor=10, dropRate=0.3)
else:
    print(
        'Specified model not recognized. Options are: resnet18 and wideresnet')
    
# Setup loss
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
criterion.__init__(reduce=False)

# Setup optimizer
if args['optimizer'] == 'adam':
    model_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
elif args['optimizer'] == 'sgd':
    model_optimizer = torch.optim.SGD(
        model.parameters(),
        lr=args['learning_rate'],
        momentum=0.9,
        nesterov=True,
        weight_decay=5e-4)
    scheduler = MultiStepLR(
        model_optimizer, milestones=[60, 120, 160], gamma=0.2)
else:
    print('Specified optimizer not recognized. Options are: adam and sgd')

c:\Users\tony\miniconda3\envs\HW3\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


### Train

In [36]:
example_stats = {}

best_acc = 0
elapsed_time = 0
for epoch in range(args['epochs']):
    start_time = time.time()

    train(args, model, device, train_dataset, model_optimizer, epoch,
          example_stats)
    test(epoch, model, device, example_stats)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d' % (get_hms(elapsed_time)))

    # Update optimizer step
    if args['optimizer'] == 'sgd':
        scheduler.step(epoch)

    # Save the stats dictionary
    fname = os.path.join(args['output_dir'], save_fname)
    with open(fname + "__stats_dict.pkl", "wb") as f:
        pickle.dump(example_stats, f)

    # Log the best train and test accuracy so far
    with open(fname + "__best_acc.txt", "w") as f:
        f.write('train test \n')
        f.write(str(max(example_stats['train'][1])))
        f.write(' ')
        f.write(str(max(example_stats['test'][1])))



=> Training Epoch #0
| Epoch [  0/ 10] Iter[391/391]		Loss: 1.7074 Acc@1: 28.998%
| Validation Epoch #0			Loss: 1.8637 Acc@1: 37.95%
| Saving Best model...			Top1 = 37.95%
| Elapsed time : 0:03:02


c:\Users\tony\miniconda3\envs\HW3\Lib\site-packages\torch\optim\lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)



=> Training Epoch #1
| Epoch [  1/ 10] Iter[391/391]		Loss: 1.2506 Acc@1: 46.856%
| Validation Epoch #1			Loss: 1.2659 Acc@1: 51.25%
| Saving Best model...			Top1 = 51.25%
| Elapsed time : 0:06:16

=> Training Epoch #2
| Epoch [  2/ 10] Iter[391/391]		Loss: 1.0101 Acc@1: 58.726%
| Validation Epoch #2			Loss: 1.1088 Acc@1: 56.04%
| Saving Best model...			Top1 = 56.04%
| Elapsed time : 0:09:33

=> Training Epoch #3
| Epoch [  3/ 10] Iter[391/391]		Loss: 0.9436 Acc@1: 67.298%
| Validation Epoch #3			Loss: 1.2252 Acc@1: 60.06%
| Saving Best model...			Top1 = 60.06%
| Elapsed time : 0:12:41

=> Training Epoch #4
| Epoch [  4/ 10] Iter[391/391]		Loss: 0.6884 Acc@1: 72.992%
| Validation Epoch #4			Loss: 0.9292 Acc@1: 69.70%
| Saving Best model...			Top1 = 69.70%
| Elapsed time : 0:15:50

=> Training Epoch #5
| Epoch [  5/ 10] Iter[391/391]		Loss: 0.5614 Acc@1: 77.270%
| Validation Epoch #5			Loss: 0.7216 Acc@1: 71.08%
| Saving Best model...			Top1 = 71.08%
| Elapsed time : 0:19:01

=> Traini

In [37]:
train_dataset.targets

[1,
 8,
 3,
 1,
 8,
 2,
 4,
 0,
 5,
 5,
 7,
 5,
 0,
 1,
 5,
 6,
 0,
 2,
 1,
 9,
 5,
 6,
 3,
 3,
 1,
 8,
 3,
 0,
 8,
 3,
 1,
 2,
 0,
 0,
 9,
 0,
 7,
 7,
 0,
 6,
 4,
 6,
 7,
 0,
 8,
 6,
 2,
 2,
 3,
 1,
 3,
 4,
 4,
 9,
 5,
 9,
 4,
 1,
 2,
 6,
 8,
 8,
 3,
 7,
 8,
 6,
 4,
 4,
 2,
 3,
 2,
 0,
 3,
 7,
 4,
 7,
 7,
 1,
 7,
 8,
 1,
 0,
 2,
 2,
 2,
 7,
 4,
 2,
 3,
 0,
 0,
 4,
 5,
 8,
 5,
 6,
 1,
 6,
 1,
 0,
 4,
 4,
 6,
 8,
 6,
 6,
 9,
 0,
 0,
 9,
 9,
 0,
 8,
 2,
 5,
 0,
 6,
 6,
 3,
 7,
 1,
 4,
 5,
 3,
 0,
 4,
 7,
 6,
 2,
 2,
 1,
 1,
 4,
 3,
 0,
 7,
 9,
 2,
 4,
 9,
 9,
 8,
 1,
 9,
 0,
 9,
 4,
 4,
 0,
 9,
 0,
 3,
 9,
 5,
 5,
 2,
 9,
 9,
 5,
 0,
 6,
 3,
 0,
 1,
 4,
 5,
 8,
 1,
 3,
 3,
 8,
 9,
 4,
 4,
 7,
 5,
 8,
 9,
 8,
 9,
 0,
 8,
 1,
 6,
 9,
 3,
 1,
 5,
 8,
 8,
 4,
 7,
 4,
 7,
 5,
 2,
 3,
 6,
 5,
 4,
 6,
 1,
 6,
 7,
 4,
 4,
 4,
 1,
 3,
 4,
 8,
 4,
 0,
 9,
 9,
 7,
 9,
 4,
 3,
 6,
 8,
 6,
 0,
 3,
 9,
 0,
 5,
 8,
 3,
 1,
 6,
 9,
 0,
 6,
 4,
 2,
 1,
 5,
 9,
 2,
 1,
 0,
 2,
 2,
 0,
 0,
 6,
 3,
 7,
 3,
